This is to test helpers.scan() in ES>5 or scrolling using 'scan' search_type in ES2.x

In [9]:
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
es = Elasticsearch([{'host':'cl-analytics.mwt2.org', 'port':9200}],
                   send_get_body_as = 'POST'     # to be passed to Transport class
                   )
triumf = ["ANALY_TRIUMF"]
source = ['pandaid','wall_time','cpuconsumptiontime']
q = {
      "_source": source,
      "query": {
        "constant_score": {
          "filter": {
            "bool": {
              "must": [
                {"range": {"endtime": {"gte":"now-2h/h", "lte":"now"}}},
                {"terms": {"computingsite": triumf}},
                {"terms": {"jobstatus": ["finished", "failed"]}}
              ]
            }
          }
        }
      }
    }

index = 'jobs_archive_2016-12*'

#sc = scan(es, query=q, index=index)
#for s in sc:
#    print(s['_source'])
        
print(elasticsearch.__version__)
if elasticsearch.__version__[0] >= 5:
    # use helpers.scan() method
    sc = scan(es, query=q, index=index)
    for s in sc:
        print(s['_source'])
else:
    # Initialize the scroll
    page = es.search(index = index,
                     body = q,
                     search_type='scan',
                     scroll = u'5m',
                     timeout= u'10m'
                    )
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']
    # start scrolling
    while (scroll_size > 0):
        page = es.scroll(scroll_id=sid, scroll=u'5m', 
                         params={'request_timeout':600})
        # Update the scroll ID
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        # Do something with the obtained page
        for doc in page['hits']['hits']:
            print(doc['_source'])


(5, 0, 1)
{'wall_time': 1191, 'pandaid': 3142526743, 'cpuconsumptiontime': 866}
{'wall_time': 829, 'pandaid': 3142530726, 'cpuconsumptiontime': 569}
{'wall_time': 2612, 'pandaid': 3142500521, 'cpuconsumptiontime': 0}
{'wall_time': 2381, 'pandaid': 3142467148, 'cpuconsumptiontime': 1927}
{'wall_time': 276, 'pandaid': 3142535937, 'cpuconsumptiontime': 45}
{'wall_time': 287, 'pandaid': 3142540203, 'cpuconsumptiontime': 35}
{'wall_time': 282, 'pandaid': 3142540271, 'cpuconsumptiontime': 44}
{'wall_time': 288, 'pandaid': 3142540135, 'cpuconsumptiontime': 38}
{'wall_time': 1865, 'pandaid': 3142530071, 'cpuconsumptiontime': 1565}
{'wall_time': 2599, 'pandaid': 3142518671, 'cpuconsumptiontime': 0}
{'wall_time': 719, 'pandaid': 3142530746, 'cpuconsumptiontime': 507}
{'wall_time': 235, 'pandaid': 3142547090, 'cpuconsumptiontime': 35}
{'wall_time': 221, 'pandaid': 3142540843, 'cpuconsumptiontime': 44}
{'wall_time': 356, 'pandaid': 3142498935, 'cpuconsumptiontime': 39}
{'wall_time': 412, 'pandaid'